In [ ]:
"""
This file is intended for training a logistic regression module to catch
the sentiments of comments
"""

In [5]:
## Import Modules
import pandas as pd

In [8]:
## Read Data
with open("IMDB Dataset.csv", "r", encoding="utf-8") as data:
    reviews: DataFrame = pd.read_csv(data)
    print(reviews)

                                                  review sentiment
0      One of the other reviewers has mentioned that ...  positive
1      A wonderful little production. <br /><br />The...  positive
2      I thought this was a wonderful way to spend ti...  positive
3      Basically there's a family where a little boy ...  negative
4      Petter Mattei's "Love in the Time of Money" is...  positive
...                                                  ...       ...
49995  I thought this movie did a down right good job...  positive
49996  Bad plot, bad dialogue, bad acting, idiotic di...  negative
49997  I am a Catholic taught in parochial elementary...  negative
49998  I'm going to have to disagree with the previou...  negative
49999  No one expects the Star Trek movies to be high...  negative

[50000 rows x 2 columns]


In [ ]:
## Process Data

In [ ]:
## Split Data

In [ ]:
## Train Model

In [ ]:
## Test Model

In [ ]:
## Save Model